# Predicting Salaries of MLB Players

- We found a dataset on kaggle.com of MLB position player statistics and salary data (adjusted for inflation) for 1985-2016. (https://www.kaggle.com/datasets/andrewdecker/hitters-salary-adjusted-to-inflation)

- The dataset comprises **15023 observations with 29 features (4 ID features, 2 salary features, 4 fielding features, 19 offensive features)**. 

- We focused on **ADJ Salary** (salary column adjusted for inflation) as our dependent variable.

- After cleaning the dataset, we had **15014 observations, 25 independent variables, 1 dependent variable**.

- Based on a suggestion in a Moneyball-themed post on Medium.com, we transformed the ADJ Salary column into its natural logarithm, thereby making the histogram distribution look more like a normal, Gaussian distribution. (https://medium.com/towards-data-science/did-the-money-follow-the-ball-analyzing-the-importance-of-baseball-batting-statistics-pre-144d7d452e1f)

- Correlation matrix analysis revealed that the offensive features were more highly correlated with our dependent variable than any of the other features in the dataset, so we focused our efforts there. **GS (games started), BB (walks), RBI (runs batted in), R (runs scored), HR (home runs), and InnOuts (inning outs, a measure of game time played)** were the highest-correlated with ADJ Salary. 

- A pairplot of these offensive features didn't show an obvious linear relationship with ADJ Salary, but unfortunately quite a bit of collinearity with each other.  

- Scatter plots between each feature of interest (GS, BB, RBI, R, HR, InnOuts) and the ADJ Salary dependent variable didn't reveal any obvious linear relationship.

- Simple univariate linear regression was conducted using LinearRegression(), GradientBoostingRegressor(), RandomForestRegressor(), and statsmodels ols regressor for each feature of interest. The regressors were unable to explain more than **20% of the variance in the dependent variable**. 

- Our next step was to attempt a multivariate linear regression with all of the features of interest, using the aforementioned regressors. This time the results were marginally better: the multivariate linear regressions explain approximately **19-26% of variance in ADJ Salary**.  

- We were disappointed with these results, so we took a different approach. What if we aggregated the dataset, and took the mean values of all of a player's stats over his career (including salary)? Could a multivariate linear regression on this aggregation perform any better?

- We grouped the dataset by playerID, took the mean of every feature, and ended up with **2468 observations**.

- Correlation analysis, pairplot visualization, and scatter plots now revealed stronger correlations and linear relationships between **RBI, R, twoB (doubles)** and ADJ Salary, but again with lots of multicollinearity. 

- Simple linear regressions for each new feature of interest and our dependent variable now showed over **50% explained variance**.

- Now a return to multivariate linear regression on these new features yielded similar values, achieving **59% explained variance** on the GradientBoostingRegressor() model. (GBR1 Testing Score: 	0.5907526209843459)

- We attempted to improve the scores using ensemble methods (Ridge and ElasticNet), but they did not improve upon the previous results. 

- Nevertheless, our methods demonstrated substantial improvement in explained variance in our dependent variable.  

In [ ]:
# Import dependencies

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# EDA 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("Hitters_Adjusted_Salary.csv")
df

In [ ]:
# Drop unnecessary columns

df = df.drop(columns=["Unnamed: 0", "salary", "teamID", "lgID"], axis=1)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Make ADJ Salary into integer

df["ADJ Salary"] = df["ADJ Salary"].astype("int").round()

In [ ]:
# Drop any rows with 0 salary

df = df.loc[(df["ADJ Salary"] > 0), :]

In [ ]:
# Render the natural logarithm of the salary column

df["ADJ Salary"] = np.log(df["ADJ Salary"])

In [ ]:
df = df.rename(columns={"2B":"twoB", "3B":"threeB"})

In [ ]:
# 15014 observations, 25 independent variables, 1 dependent variable

df = df.drop_duplicates().reset_index(drop=True)
df

In [ ]:
# Look at distributions of the variables

df.hist(figsize = (15, 15))  

In [ ]:
# Correlation matrix reveals the best independent variables: RBI, BB, GS, R, HR 
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
cols = ["ADJ Salary","GS","InnOuts","PO","A","E","DP","G","AB","R",\
        "H","twoB","threeB","HR","RBI","SB","CS","BB","SO","IBB","HBP","SH","SF","GIDP"]

corr = df[cols].corr()
corr = corr.style.background_gradient(cmap='Blues')
corr

In [ ]:
# Looking for multicollinearity

sns.pairplot(df[["ADJ Salary","RBI","BB","GS","R","HR","InnOuts"]])

In [ ]:
corr = df[["GS","BB","RBI","R","HR","InnOuts"]].corr()
corr = corr.style.background_gradient(cmap='Purples')
corr

In [ ]:
# Look at scatterplots

plt.scatter(df["GS"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("GS")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(df["BB"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("BB")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(df["RBI"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("RBI")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(df["R"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("R")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(df["HR"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("HR")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(df["InnOuts"][:1000], df["ADJ Salary"][:1000])
plt.xlabel("InnOuts")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
# Perform linear regression on each candidate independent variable

def simple_LR(a_df, col_list):
            
    # Assign X and y

    X = a_df[col_list]

    # X = df.drop(columns=["ADJ Salary", "playerID"])

    y = a_df["ADJ Salary"]

    # Split the data into X_train, X_test, y_train, y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

    # Create a scaler to standardize the data

    scaler = StandardScaler()

    # Train the scaler with the X_train data.

    scaler.fit(X_train)

    # Transform X_train and X_test.

    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    LR1 = LinearRegression().fit(X_train_scaled, y_train)
    GBR1 = GradientBoostingRegressor().fit(X_train_scaled, y_train)
    RFR1 = RandomForestRegressor().fit(X_train_scaled, y_train)

    LR1_pred = LR1.predict(X_test)
    GBR1_pred = GBR1.predict(X_test)
    RFR1_pred = RFR1.predict(X_test)

    LR1_mse = mean_squared_error(y_test, LR1_pred)
    GBR1_mse = mean_squared_error(y_test, GBR1_pred)
    RFR1_mse = mean_squared_error(y_test, RFR1_pred)

    LR1_r2 = r2_score(y_test, LR1_pred)
    GBR1_r2 = r2_score(y_test, GBR1_pred)
    RFR1_r2 = r2_score(y_test, RFR1_pred)

    # Score the regression models

    print(f"LR1 Training Score: \t\t{LR1.score(X_train_scaled, y_train)}")
    print(f"LR1 Testing Score: \t{LR1.score(X_test_scaled, y_test)}")
    print(f"LR1 r2: \t\t\t{LR1_r2}")
    print(f"LR1 mse: \t\t\t{LR1_mse}\n")

    print(f"GBR1 Training Score: \t\t{GBR1.score(X_train_scaled, y_train)}")
    print(f"GBR1 Testing Score: \t{GBR1.score(X_test_scaled, y_test)}")
    print(f"GBR1 r2: \t\t\t{GBR1_r2}")
    print(f"GBR1 mse: \t\t\t{GBR1_mse}\n")

    print(f"RFR1 Training Score: \t\t{RFR1.score(X_train_scaled, y_train)}")
    print(f"RFR1 Testing Score: \t{RFR1.score(X_test_scaled, y_test)}")
    print(f"RFR1 r2: \t\t\t{RFR1_r2}")
    print(f"RFR1 mse: \t\t\t{RFR1_mse}\n")

    formula = f'y ~ {" + ".join(c for c in col_list)}'

    LR1_stats = smf.ols(formula=formula, data=X).fit()

    print(LR1_stats.summary())

In [ ]:
simple_LR(df, ["RBI"])

In [ ]:
simple_LR(df, ["GS"])

In [ ]:
simple_LR(df, ["BB"])

In [ ]:
simple_LR(df, ["R"])

In [ ]:
simple_LR(df, ["HR"])

In [ ]:
simple_LR(df, ["InnOuts"])

In [ ]:
df.to_csv("first_predictions_df.csv")

## Results:

- all the highly correlated independent vars are also correlated with each other = multicollinearity!

## First attempt at multivariate linear regression 

In [ ]:
# Assign X and y

X = df[["RBI", "BB", "GS", "R", "HR"]]

y = df["ADJ Salary"]

# Split the data into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

# Create a scaler to standardize the data

scaler = StandardScaler()

# Train the scaler with the X_train data.

scaler.fit(X_train)

# Transform X_train and X_test.

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

LR1 = LinearRegression().fit(X_train_scaled, y_train)
GBR1 = GradientBoostingRegressor().fit(X_train_scaled, y_train)
RFR1 = RandomForestRegressor().fit(X_train_scaled, y_train)

LR1_pred = LR1.predict(X_test)
GBR1_pred = GBR1.predict(X_test)
RFR1_pred = RFR1.predict(X_test)

LR1_mse = mean_squared_error(y_test, LR1_pred)
GBR1_mse = mean_squared_error(y_test, GBR1_pred)
RFR1_mse = mean_squared_error(y_test, RFR1_pred)

LR1_r2 = r2_score(y_test, LR1_pred)
GBR1_r2 = r2_score(y_test, GBR1_pred)
RFR1_r2 = r2_score(y_test, RFR1_pred)

# Score the regression models

print(f"LR1 Training Score: \t\t{LR1.score(X_train_scaled, y_train)}")
print(f"LR1 Testing Score: \t{LR1.score(X_test_scaled, y_test)}")
print(f"LR1 r2: \t\t\t{LR1_r2}")
print(f"LR1 mse: \t\t\t{LR1_mse}\n")

print(f"GBR1 Training Score: \t\t{GBR1.score(X_train_scaled, y_train)}")
print(f"GBR1 Testing Score: \t{GBR1.score(X_test_scaled, y_test)}")
print(f"GBR1 r2: \t\t\t{GBR1_r2}")
print(f"GBR1 mse: \t\t\t{GBR1_mse}\n")

print(f"RFR1 Training Score: \t\t{RFR1.score(X_train_scaled, y_train)}")
print(f"RFR1 Testing Score: \t{RFR1.score(X_test_scaled, y_test)}")
print(f"RFR1 r2: \t\t\t{RFR1_r2}")
print(f"RFR1 mse: \t\t\t{RFR1_mse}\n")


LR1_stats = smf.ols(formula = "y ~ RBI + BB + GS + R + HR", data=X).fit()

LR1_stats.summary()

## Results: 
- the multivariate linear regressions explain approximately 19-26% of variance
- we can do better!

## Second attempt at multivariate linear regression ... a more savvy approach this time 

In [ ]:
# I'm now aggregating the data across players' careers, taking the mean of all variables

df = pd.read_csv("first_predictions_df.csv", index_col="Unnamed: 0")

agg_df = df.groupby(["playerID"]).mean()
agg_df.to_csv("second_predictions_df.csv")
agg_df

In [ ]:
agg_df.duplicated().sum()

In [ ]:
# Look at distributions of the variables

agg_df.hist(figsize = (15, 15))  

In [ ]:
# Correlation matrix reveals the best independent variables: RBI, H, R, 2B

cols = ["ADJ Salary","GS","InnOuts","PO","A","E","DP","G","AB","R",\
        "H","twoB","threeB","HR","RBI","SB","CS","BB","SO","IBB","HBP","SH","SF","GIDP"]

corr = agg_df[cols].corr()
corr = corr.style.background_gradient(cmap='Purples')
corr

In [ ]:
# Looking for multicollinearity

sns.pairplot(agg_df[["ADJ Salary","RBI", "R", "twoB"]])

In [ ]:
# Lots of multicollinearity, but all these vars plot a linear relationship with ADJ Salary

corr = agg_df[["ADJ Salary","RBI", "R", "twoB"]].corr()
corr = corr.style.background_gradient(cmap='Purples')
corr

In [ ]:
plt.scatter(agg_df["RBI"], agg_df["ADJ Salary"])
plt.xlabel("RBI")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(agg_df["R"], agg_df["ADJ Salary"])
plt.xlabel("R")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
plt.scatter(agg_df["twoB"], agg_df["ADJ Salary"])
plt.xlabel("twoB")
plt.ylabel("ADJ Salary")
plt.show()

In [ ]:
simple_LR(agg_df, ["RBI"])

In [ ]:
simple_LR(agg_df, ["R"])

In [ ]:
simple_LR(agg_df, ["twoB"])

## Let's try multivariate linear regression again, this time on the aggregated dataset

In [ ]:
agg_df = pd.read_csv("second_predictions_df.csv", index_col="playerID")
agg_df

# Assign X and y

X = agg_df[["RBI", "R", "twoB"]]

y = agg_df["ADJ Salary"]

# Split the data into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

# Create a scaler to standardize the data

scaler = StandardScaler()

# Train the scaler with the X_train data.

scaler.fit(X_train)

# Transform X_train and X_test.

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

LR1 = LinearRegression().fit(X_train_scaled, y_train)
GBR1 = GradientBoostingRegressor().fit(X_train_scaled, y_train)
RFR1 = RandomForestRegressor().fit(X_train_scaled, y_train)

LR1_pred = LR1.predict(X_test)
GBR1_pred = GBR1.predict(X_test)
RFR1_pred = RFR1.predict(X_test)

LR1_mse = mean_squared_error(y_test, LR1_pred)
GBR1_mse = mean_squared_error(y_test, GBR1_pred)
RFR1_mse = mean_squared_error(y_test, RFR1_pred)

LR1_r2 = r2_score(y_test, LR1_pred)
GBR1_r2 = r2_score(y_test, GBR1_pred)
RFR1_r2 = r2_score(y_test, RFR1_pred)

# Score the regression models

print(f"LR1 Training Score: \t\t{LR1.score(X_train_scaled, y_train)}")
print(f"LR1 Testing Score: \t{LR1.score(X_test_scaled, y_test)}")
print(f"LR1 r2: \t\t\t{LR1_r2}")
print(f"LR1 mse: \t\t\t{LR1_mse}\n")

print(f"GBR1 Training Score: \t\t{GBR1.score(X_train_scaled, y_train)}")
print(f"GBR1 Testing Score: \t{GBR1.score(X_test_scaled, y_test)}")
print(f"GBR1 r2: \t\t\t{GBR1_r2}")
print(f"GBR1 mse: \t\t\t{GBR1_mse}\n")

print(f"RFR1 Training Score: \t\t{RFR1.score(X_train_scaled, y_train)}")
print(f"RFR1 Testing Score: \t{RFR1.score(X_test_scaled, y_test)}")
print(f"RFR1 r2: \t\t\t{RFR1_r2}")
print(f"RFR1 mse: \t\t\t{RFR1_mse}\n")

# LR1_stats = smf.ols(formula = 'y ~ yearID + GS + InnOuts + PO + A + E + DP + G + AB + R + H +\
# twoB + threeB + HR + RBI + SB + CS + BB + SO + IBB + HBP + SH + SF + GIDP', data=X).fit()

LR1_stats = smf.ols(formula = "y ~ RBI + R + twoB", data=X).fit()
                    
LR1_stats.summary()

## Results: 
- the linear regressions on the aggregated data now explain nearly 60% of variance!

# Linear Regression Ensemble Methods

In [ ]:
# LinearRegression()

agg_df = pd.read_csv("second_predictions_df.csv", index_col="playerID")
agg_df

# Assign X and y

X = agg_df[["RBI","R","twoB"]]

y = agg_df["ADJ Salary"]

# Split the data into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

# Create a scaler to standardize the data

scaler = StandardScaler()

# Train the scaler with the X_train data.

scaler.fit(X_train)

# Transform X_train and X_test.

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression().fit(X_train_scaled, y_train)

# Score the model

print(f"STDSCALER Linear Regression Score: {model.score(X_train_scaled, y_train)}")
print(f"STDSCALER Linear Regression Score: {model.score(X_test_scaled, y_test)}")

In [ ]:
agg_df = pd.read_csv("second_predictions_df.csv", index_col="playerID")
agg_df

# Assign X and y

X = agg_df[["RBI","R","twoB"]]

y = agg_df["ADJ Salary"]

# Split the data into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

# Create a scaler to standardize the data

scaler = StandardScaler()

# Train the scaler with the X_train data.

scaler.fit(X_train)

# Transform X_train and X_test.

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge_reg = Ridge().fit(X_train, y_train)

predicted = ridge_reg.predict(X_test_scaled)
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"MSE: {mse}")
print(f"R2: {r2}") 

# Score the model

print(f"Ridge Regression Score: {ridge_reg.score(X_train_scaled, y_train)}")
print(f"Ridge Regression Score: {ridge_reg.score(X_test_scaled, y_test)}")

plt.bar(np.arange(len(ridge_reg.coef_)), ridge_reg.coef_)
plt.title(f'Ridge Regression coefficient plot')
plt.show()     

sel = SelectFromModel(ridge_reg)
sel.fit(X_train_scaled, y_train)
SelectFromModel(estimator=Ridge())

X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)

scaler = StandardScaler().fit(X_selected_train)

X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

new_ridge_reg = LinearRegression().fit(X_selected_train_scaled, y_train)
print(f"New ridge regression score: {new_ridge_reg.score(X_selected_test_scaled, y_test)}")

In [ ]:
agg_df = pd.read_csv("second_predictions_df.csv", index_col="playerID")
agg_df

# Assign X and y


X = agg_df[["RBI","R","twoB"]]

y = agg_df["ADJ Salary"]

# Split the data into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 

# Create a scaler to standardize the data

scaler = StandardScaler()

# Train the scaler with the X_train data.

scaler.fit(X_train)

# Transform X_train and X_test.

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

elasticnet_reg = ElasticNet().fit(X_train, y_train)

predicted = elasticnet_reg.predict(X_test_scaled)
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"MSE: {mse}")
print(f"R2: {r2}") 

# Score the model

print(f"ElasticNet Regression Score: {elasticnet_reg.score(X_train_scaled, y_train)}")
print(f"ElasticNet Regression Score: {elasticnet_reg.score(X_test_scaled, y_test)}")

plt.bar(np.arange(len(elasticnet_reg.coef_)), elasticnet_reg.coef_)
plt.title(f'ElasticNet Regression coefficient plot')
plt.show()  

sel = SelectFromModel(elasticnet_reg)
sel.fit(X_train_scaled, y_train)
SelectFromModel(estimator=ElasticNet())

X_selected_train, X_selected_test, y_train, y_test = train_test_split(sel.transform(X), y, random_state=1)

scaler = StandardScaler().fit(X_selected_train)

X_selected_train_scaled = scaler.transform(X_selected_train)
X_selected_test_scaled = scaler.transform(X_selected_test)

new_elasticnet_reg = LinearRegression().fit(X_selected_train_scaled, y_train)
print(f"New linear regression score: {new_elasticnet_reg.score(X_selected_test_scaled, y_test)}")